In [1]:
import sys, os, time
import shutil
%pdb
# %load_ext autoreload
# %autoreload 2
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
os.environ['FOUNDATION_SAVE_DIR'] = '/is/ei/fleeb/workspace/chome/trained_nets'
os.environ['FOUNDATION_DATA_DIR'] = '/is/ei/fleeb/workspace/local_data'
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
import torchvision

from torch.utils.data import Dataset, DataLoader, TensorDataset
import gym
import numpy as np
# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import imageio
import seaborn as sns
# plt.switch_backend('Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from bisect import bisect_left

# from IPython.display import HTML, Image

#from foundation.util import replicate, Cloner

from hybrid import get_model, get_data
from analyze import *

from tqdm import tqdm_notebook as tqdm

# plt.ioff()

np.set_printoptions(linewidth=120, suppress=True)

Automatic pdb calling has been turned ON


In [2]:
media_dir = '/is/ei/fleeb/workspace/media/hybrid/results'

In [3]:
M = Hybrid_Controller(media_dir).sort_by('date')

Found 123 runs


In [4]:
M.clear_filters()
pass

In [5]:
M.prep_info(name='model.pth.tar')

M.load_into('eval.pth.tar')

Selecting checkpoint: last


/is/ei/fleeb/workspace/foundation/foundation/train/config.py:42: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


Loaded configs


In [15]:
def _get_stats(run):
    if 'eval' in run:
        run.stats = {
            'FID-prior': run.eval['FID-prior'],
            
            
#             'DCI-inf': run.eval['DCI']['informativeness_test'],
#             'DCI-cmp': run.eval['DCI']['completeness'],
        }
        
        if 'FID-hyb' in run.eval and run.eval['FID-hyb'] is not None:
            run.stats.update({
                'FID-hyb': run.eval['FID-hyb'],
                'FID-rec': run.eval['FID-rec'],
            })
        
        if 'IRS' in run.eval and run.eval['IRS'] is not None:
            run.stats.update({
                
                'IRS': run.eval['IRS']['IRS'],
                'DCI-dis': run.eval['DCI']['disentanglement'],
            })
M.map(_get_stats, safe=True)

M.filter(lambda r: 'stats' in r and len(r.stats) and 'FID-hyb' in r.stats)
pass

M.store_selection('done')


In [16]:
M.show('all')

  0) 3dshapes-dvae_0122-5969012-01_200110-140610_ckpt100
	Checkpoint: 100
	 FID-prior : 59.65
	   FID-hyb : 58.57
	   FID-rec : 42.62
	       IRS : 0.54
	   DCI-dis : 0.22
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)
	model.prob - 1 (0.5)

  1) 3dshapes-dvae_0122-5969012-02_200110-140616_ckpt100
	Checkpoint: 100
	 FID-prior : 60.49
	   FID-hyb : 53.13
	   FID-rec : 42.52
	       IRS : 0.57
	   DCI-dis : 0.39
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)

  2) 3dshapes-dwae_0094-5966044-04_200108-175514_ckpt100
	Checkpoint: 100
	 FID-prior : 37.17
	   FID-hyb : 37.06
	   FID-rec : 24.45
	       IRS : 0.57
	   DCI-dis : 0.26
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)
	model.prob - 1 (0.5)

  3) 3dshapes-dwae_0122-5969012-00_200110-140613_ckpt100
	Checkpoint: 100
	 FID-

In [17]:
M.switch_selection('done')
len(M)

9

In [18]:
# M.invert()
# len(M)
# M.show()
# ' '.join(['_'.join(run.name.split('_')[:-1]) for run in M])

In [19]:
plt.close('all')
figaxs = {}

In [20]:
M.map(lambda r: r.meta.job[0], reduce=Counter)

Counter({'122': 4, '94': 5})

In [23]:
M.switch_selection('done')

# M.filter(lambda r: 'IRS' in r.stats)

M.sort_by(lambda runs: util.sort_by(runs, [run.stats['IRS'] for run in runs]))
# M.filter_sel(slice(-10,None,None))


# M.filter_models('fdwae')
M.filter_jobs(94, 122)
# M.show('checkpoint', 'unique', )
# M.filter(lambda r: r.stats['DCI-dis'] > 0.75)

pass

In [24]:
fig_stats = {
    'dis':['DCI-dis', 'IRS'], 
    'fid2': ['FID-rec', 'FID-hyb'],
    'fid': ['FID-prior', ],
#              'fid':['FID-rec', ],
            }
for i, (name,stats,) in enumerate(fig_stats.items()):
    if name not in figaxs:
        figaxs[name] = plt.subplots(figsize=(12,5))
    figaxs[name][1].cla()
    figaxs[name] = M.plot_stats(*stats, figax=figaxs.get(name,None))#, color='C{}'.format(i))
    plt.tight_layout()
# M.show('checkpoint', 'unique', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])
M.show('all', manuals=[lambda r: 'Generator: {}'.format(r.config.model.generator._type)])

  0) 3dshapes-dwae_0122-5969012-00_200110-140613_ckpt100
	Checkpoint: 100
	 FID-prior : 31.65
	   FID-hyb : 31.54
	   FID-rec : 25.86
	       IRS : 0.52
	   DCI-dis : 0.34
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)
	Generator: double-dec

  1) 3dshapes-dvae_0122-5969012-01_200110-140610_ckpt100
	Checkpoint: 100
	 FID-prior : 59.65
	   FID-hyb : 58.57
	   FID-rec : 42.62
	       IRS : 0.54
	   DCI-dis : 0.22
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)
	model.prob - 1 (0.5)
	Generator: double-dec

  2) 3dshapes-fdwae_0094-5966044-05_200108-175529_ckpt100
	Checkpoint: 100
	 FID-prior : 35.67
	   FID-hyb : 36.40
	   FID-rec : 25.84
	       IRS : 0.54
	   DCI-dis : 0.34
	model.generator.head._type - nn (mlp)
	model.discriminator.tail._type - nn (mlp)
	model.encoder.tail._type - nn (mlp)
	model.prob - 1 (0.5)
	Generator: double-dec

  3) 3dsha

In [32]:
run = M[0]

In [ ]:
run.config.model.generator._type

In [33]:
run.stats

{'FID-prior': 37.1679128982002,
 'FID-hyb': 37.057571602768604,
 'FID-rec': 24.448076694844502,
 'IRS': 0.5678702950047886,
 'DCI-dis': 0.26163522403866735}